# Passive degrees of freedom



Passive dofs!

In [ ]:
%run -i ../../../examples/prepare_bluesky.py # prepare the bluesky environment

from bloptools import devices, test_functions
from bloptools.bayesian import Agent


def digestion(db, uid):
    products = db[uid].table()

    for index, entry in products.iterrows():
        products.loc[index, "styblinksi-tang"] = test_functions.styblinski_tang(entry.x - 1e-1 * entry.brownian)

    return products


dofs = [
    {"device": devices.DOF(name="x"), "limits": (-5, 5), "kind": "active"},
    {"device": devices.BrownianMotion(name="brownian"), "limits": (-2, 2), "kind": "passive"},
]

tasks = [
    {"key": "styblinksi-tang", "kind": "minimize"},
]

agent = Agent(
    dofs=dofs,
    tasks=tasks,
    digestion=digestion,
    db=db,
)

RE(agent.initialize("qr", n_init=32))

agent.plot_tasks()